In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import joblib

In [2]:
# 1. Download Bitcoin Data
df = yf.download('BTC-USD', start='2014-01-01', end='2023-12-31')
df = df[['Close', 'Volume']]
df['MA20'] = df['Close'].rolling(window=20).mean()
df['MA50'] = df['Close'].rolling(window=50).mean()
df.dropna(inplace=True)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [4]:
# 2. Feature Scaling
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

In [5]:
# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [6]:
# 3. Prepare sequences
X, y = [], []
for i in range(60, len(scaled_data)):
    X.append(scaled_data[i-60:i])
    y.append(scaled_data[i, 0])  # Predict 'Close'

X, y = np.array(X), np.array(y)

In [7]:
# 4. Define LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

C:\Users\Prasanna dindi\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=10, batch_size=32)

# Save model
model.save('btc_lstm_model.h5')

Epoch 1/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - loss: 0.0196
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.0019
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.0018
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.0018
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.0015
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0017
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - loss: 0.0013
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - loss: 0.0013
Epoch 9/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 0.0015
Epoch 10/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 0.0014
